 ### 最初のインテリジェントエージェントチームを構築しよう: ADKを使った進化するウェザーボット
 
 このチュートリアルは、[Agent Development Kit](https://google.github.io/adk-docs/get-started/)の[クイックスタート例]https://google.github.io/adk-docs/get-started/quickstart/)を拡張したものです。今、あなたはより深く掘り下げ、より洗練された**ルチエージェントシステム**を構築する準備ができています。
 
 私たちは、**ウェザーボットエージェントチーム**を構築し、シンプルな基盤に高度な機能を段階的に追加していきます。天気を調べることがきる単一のエージェントから始め、次のような機能を段階的に追加します:
 
 *   異なるAIモデル（Gemini、GPT、Claude）を活用する。
 *   特定のタスク（挨拶や別れの挨拶など）に特化したサブエージェントを設計する。
 *   エージェント間のインテリジェントな委任を可能にする。
 *   永続的なセッション状態を使用してエージェントに記憶を与える。
 *   コールバックを使用して重要な安全ガードレールを実装する。
 
 **なぜウェザーボットチームなのか？**
 
 このユースケースは、一見シンプルに見えますが、複雑で現実的なエージェントアプリケーションを構築するために必要なADKのコアコンセプト探求するための実用的で親しみやすいキャンバスを提供します。インタラクションの構造化、状態の管理、安全性の確保、複数のAI「頭脳」を協させる方法を学びます。
 
 **ADKとは何か？**
 
 ADKは、大規模言語モデル（LLM）を活用したアプリケーションの開発を効率化するために設計されたPythonフレームワークです。エージェント推論し、計画し、ツールを利用し、ユーザーと動的に対話し、チーム内で効果的に協力するための強力な構成要素を提供します。
 
 **この高度なチュートリアルで習得すること:**
 
 *   ✅ **ツールの定義と使用法:** エージェントに特定の能力（データの取得など）を与えるPython関数（`tools`）を作成し、それを効果的使用する方法をエージェントに指示します。
 *   ✅ **マルチLLMの柔軟性:** LiteLLM統合を通じて、Gemini、GPT-4o、Claude Sonnetなどのさまざまな主要LLMを利用するようにエージントを設定し、各タスクに最適なモデルを選択できるようにします。
 *   ✅ **エージェントの委任と協力:** 特化したサブエージェントを設計し、チーム内で最も適切なエージェントにユーザーリクエストを自動にルーティングする（`auto flow`）を可能にします。
 *   ✅ **メモリのためのセッション状態:** `Session State`と`ToolContext`を利用して、エージェントが会話のターンを超えて情報を記し、よりコンテキストに基づいたインタラクションを実現します。
 *   ✅ **コールバックによる安全ガードレール:** `before_model_callback`と`before_tool_callback`を実装して、事前定義されたルーに基づいてリクエストやツールの使用を検査、修正、またはブロックし、アプリケーションの安全性と制御を強化します。
 
 **最終状態の期待:**
 
 このチュートリアルを完了することで、機能的なマルチエージェントウェザーボットシステムを構築することができます。このシステムは、天気情報を提供するだけでなく、会話の礼儀を処理し、最後に確認した都市を記憶し、定義された安全境界内で動作し、すべてADKを使用して調整されます。
 
 **前提条件:**
 
 *   ✅ **Pythonプログラミングの確かな理解。**
 *   ✅ **大規模言語モデル（LLM）、API、およびエージェントの概念に精通していること。**
 *   ❗ **重要: ADKクイックスタートチュートリアルの完了、またはADKの基本（エージェント、ランナー、セッションサービス、基本的なツーの使用法）に関する同等の基礎知識。** このチュートリアルはこれらの概念に直接基づいています。
 *   ✅ **使用するLLMのAPIキー**（例: Gemini用のGoogle AI Studio、OpenAIプラットフォーム、Anthropicコンソール）。
 
 **エージェントチームを構築する準備はできましたか？さあ、始めましょう！**

In [1]:
# @title Step 0: Setup and Installation
# Install ADK and LiteLLM for multi-model support

!pip install google-adk -q
!pip install litellm -q

print("Installation complete.")

Installation complete.


In [5]:
# @title Import necessary libraries
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # For creating message Content/Parts

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("Libraries imported.")

Libraries imported.


In [6]:
# @title Configure API Keys using dotenv

# .envファイルから環境変数を読み込むためにdotenvをインポート
from dotenv import load_dotenv

# .envファイルから環境変数を読み込む
load_dotenv()

# 環境変数からAPIキーを取得
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# 環境変数に設定
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY if GOOGLE_API_KEY else "YOUR_GOOGLE_API_KEY"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY if OPENAI_API_KEY else "YOUR_OPENAI_API_KEY"
os.environ["ANTHROPIC_API_KEY"] = ANTHROPIC_API_KEY if ANTHROPIC_API_KEY else "YOUR_ANTHROPIC_API_KEY"

# --- APIキーの確認 ---
print("API Keys Set:")
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"OpenAI API Key set: {'Yes' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'YOUR_OPENAI_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")
print(f"Anthropic API Key set: {'Yes' if os.environ.get('ANTHROPIC_API_KEY') and os.environ['ANTHROPIC_API_KEY'] != 'YOUR_ANTHROPIC_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

# ADKがVertex AIではなく直接APIキーを使用するように設定
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"


# @markdown **セキュリティに関する注意:** APIキーは安全に管理することがベストプラクティスです（例：Colab Secretsや環境変数を使用）。.envファイルを使用することで、APIキーをコードに直接記述せずに管理できます。

API Keys Set:
Google API Key set: Yes
OpenAI API Key set: Yes
Anthropic API Key set: Yes


In [12]:
# --- Define Model Constants for easier use ---

MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

# Note: Specific model names might change. Refer to LiteLLM/Provider documentation.
MODEL_GPT_o3_MINI = "openai/o3-mini-2025-01-31"
MODEL_CLAUDE_SONNET = "anthropic/claude-3-sonnet-20240229"


print("\nEnvironment configured.")


Environment configured.


 ---
 
 ## ステップ 1: 最初のエージェント \- 基本的な天気検索
 
 天気ボットの基本的なコンポーネントを構築することから始めましょう：特定のタスク（天気情報の検索）を実行できる単一のエージェントす。これには、次の2つの主要な部分を作成する必要があります：
 
 1. **ツール:** エージェントに天気データを取得する*能力*を与えるPython関数。  
 2. **エージェント:** ユーザーのリクエストを理解し、天気ツールを持っていることを認識し、いつどのようにそれを使用するかを決定するAの「頭脳」。
 
 ---
 
 **1\. ツールの定義 (`get_weather`)**
 
 ADKでは、**ツール**はエージェントにテキスト生成を超えた具体的な能力を与える構成要素です。これらは通常、APIの呼び出し、データベーの照会、計算の実行など、特定のアクションを実行する通常のPython関数です。
 
 最初のツールは*模擬*天気レポートを提供します。これにより、外部APIキーをまだ必要とせずにエージェント構造に集中できます。後で、この擬関数を実際の天気サービスを呼び出す関数に簡単に置き換えることができます。
 
 **重要な概念：ドキュメント文字列（Docstrings）が重要です！** エージェントのLLMは関数の**ドキュメント文字列**に大きく依存して以を理解します：
 
 * ツールが*何を*するのか。  
 * *いつ*使用するのか。  
 * *どの引数*が必要か（`city: str`）。  
 * *どのような情報*を返すのか。
 
 **ベストプラクティス：** ツールに対して明確で、説明的で、正確なドキュメント文字列を書きましょう。これはLLMがツールを正しく使用すために不可欠です。

In [13]:
# @title get_weatherツールを定義する
def get_weather(city: str) -> dict:
    """指定された都市の現在の天気予報を取得します。

    引数:
        city (str): 都市の名前（例: "New York", "London", "Tokyo"）。
        都市名は英語にしてください。

    戻り値:
        dict: 天気情報を含む辞書。
            'status' キー（'success' または 'error'）を含みます。
            'success' の場合、天気の詳細を含む 'report' キーを含みます。
            'error' の場合、'error_message' キーを含みます。
    """
    print(f"--- ツール: get_weather が都市: {city} のために呼び出されました ---") # ツール実行のログ
    city_normalized = city.lower().replace(" ", "") # 基本的な正規化

    # 模擬天気データ
    mock_weather_db = {
        "newyork": {"status": "success", "report": "The weather in New York is sunny with a temperature of 25°C."},
        "london": {"status": "success", "report": "It's cloudy in London with a temperature of 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo is experiencing light rain and a temperature of 18°C."},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"申し訳ありません。'{city}'の天気情報はありません。"}

# ツール使用例（オプションのテスト）
print(get_weather("New York"))
print(get_weather("Paris"))

--- ツール: get_weather が都市: New York のために呼び出されました ---
{'status': 'success', 'report': 'The weather in New York is sunny with a temperature of 25°C.'}
--- ツール: get_weather が都市: Paris のために呼び出されました ---
{'status': 'error', 'error_message': "申し訳ありません。'Paris'の天気情報はありません。"}


 ---
 
 **2\. エージェントの定義 (`weather_agent`)**
 
 次に、**エージェント**自体を作成しましょう。ADKにおける`Agent`は、ユーザー、LLM、利用可能なツール間の相互作用を調整します。
 
 以下のような重要なパラメータで設定します：
 
 * `name`: このエージェントの一意の識別子（例：「weather\_agent\_v1」）。  
 * `model`: 使用するLLMを指定します（例：`MODEL_GEMINI_2_5_PRO`）。特定のGeminiモデルから始めます。  
 * `description`: エージェントの全体的な目的の簡潔な要約。これは後で他のエージェントが*このエージェント*にタスクを委任するかどうを決定する際に重要になります。  
 * `instruction`: LLMの振る舞い方、ペルソナ、目標、特に割り当てられた`tools`を*どのように、いつ*使用するかについての詳細なガイダス。  
 * `tools`: エージェントが使用を許可されている実際のPythonツール関数を含むリスト（例：`[get_weather]`）。
 
 **ベストプラクティス：** 明確で具体的な`instruction`プロンプトを提供しましょう。指示が詳細であればあるほど、LLMは自分の役割とツルの効果的な使用方法をより理解できます。必要に応じてエラー処理について明示的に記述してください。
 
 **ベストプラクティス：** 説明的な`name`と`description`の値を選びましょう。これらはADK内部で使用され、自動委任（後で説明）などの能に不可欠です。

In [14]:
# @title 天気エージェントの定義
# 前に定義したモデル定数の1つを使用
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH # Geminiから始める

weather_agent = Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL, # GeminiのモデルまたはLiteLlmオブジェクトを指定可能
    description="特定の都市の天気情報を提供します。",
    instruction="あなたは役に立つ天気アシスタントです。"
                "ユーザーが特定の都市の天気を尋ねる場合、"
                "'get_weather'ツールを使用して情報を取得してください。"
                "ツールがエラーを返した場合は、丁寧にユーザーに伝えてください。"
                "ツールが成功した場合は、天気レポートを明確に提示してください。",
    tools=[get_weather], # 関数を直接渡す
)

print(f"エージェント '{weather_agent.name}' がモデル '{AGENT_MODEL}' を使用して作成されました。")

エージェント 'weather_agent_v1' がモデル 'gemini-2.0-flash' を使用して作成されました。


 ---
 
 **3\. Runnerとセッションサービスの設定**
 
 会話を管理しエージェントを実行するために、さらに2つのコンポーネントが必要です：
 
 * `SessionService`: 異なるユーザーやセッションの会話履歴と状態を管理する役割を担います。`InMemorySessionService`はすべてをメリに保存するシンプルな実装で、テストやシンプルなアプリケーションに適しています。交換されたメッセージを追跡します。状態の永続化についはステップ4でさらに詳しく説明します。  
 * `Runner`: 対話フローを調整するエンジンです。ユーザー入力を受け取り、適切なエージェントにルーティングし、エージェントのロジックに基づいてLLMやツールの呼び出しを管理し、`SessionService`を通じてセッションの更新を処理し、対話の進行状況を表すイベントを生成します。

In [15]:
# @title セッションサービスとランナーの設定

# --- セッション管理 ---
# 重要概念: SessionServiceは会話履歴と状態を保存します。
# InMemorySessionServiceはこのチュートリアル用の単純な非永続的ストレージです。
session_service = InMemorySessionService()

# 対話コンテキストを識別するための定数を定義
APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # 簡略化のために固定IDを使用

# 会話が行われる特定のセッションを作成
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"セッションが作成されました: アプリ='{APP_NAME}', ユーザー='{USER_ID}', セッション='{SESSION_ID}'")

# --- ランナー ---
# 重要概念: Runnerはエージェント実行ループを調整します。
runner = Runner(
    agent=weather_agent, # 実行したいエージェント
    app_name=APP_NAME,   # 実行をアプリに関連付ける
    session_service=session_service # セッションマネージャーを使用
)
print(f"エージェント '{runner.agent.name}' のランナーが作成されました。")

セッションが作成されました: アプリ='weather_tutorial_app', ユーザー='user_1', セッション='session_001'
エージェント 'weather_agent_v1' のランナーが作成されました。


 ---
 
 **4\. エージェントとの対話**
 
 エージェントにメッセージを送信し、その応答を受け取る方法が必要です。LLM呼び出しやツールの実行には時間がかかる可能性があるため、ADの`Runner`は非同期で動作します。
 
 以下のような`async`ヘルパー関数（`call_agent_async`）を定義します：
 
 1. ユーザークエリ文字列を受け取ります。  
 2. それをADK `Content`形式にパッケージ化します。  
 3. ユーザー/セッションコンテキストと新しいメッセージを提供して`runner.run_async`を呼び出します。  
 4. ランナーから生成される**イベント**を反復処理します。イベントはエージェントの実行ステップ（例：ツール呼び出し要求、ツール結果信、中間LLM思考、最終応答）を表します。  
 5. `event.is_final_response()`を使用して**最終応答**イベントを識別し、表示します。
 
 **なぜ`async`なのか？** LLMや潜在的なツール（外部APIなど）とのやり取りはI/O制約のある操作です。`asyncio`を使用することで、実行をブロックすることなくこれらの操作を効率的に処理できます。

In [16]:
# @title エージェント対話関数の定義

from google.genai import types # メッセージのContent/Partsを作成するため

async def call_agent_async(query: str, runner, user_id, session_id):
   """エージェントにクエリを送信し、最終応答を表示します。"""
   print(f"\n>>> ユーザークエリ: {query}")

   # ユーザーのメッセージをADK形式で準備
   content = types.Content(role='user', parts=[types.Part(text=query)])

   final_response_text = "エージェントは最終応答を生成しませんでした。" # デフォルト

   # 重要概念: run_asyncはエージェントロジックを実行し、イベントを生成します。
   # 最終回答を見つけるためにイベントを反復処理します。
   async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # 以下の行のコメントを解除すると、実行中の*すべての*イベントを確認できます
      # print(f"  [イベント] 作成者: {event.author}, タイプ: {type(event).__name__}, 最終: {event.is_final_response()}, 内容: {event.content}")

      # 重要概念: is_final_response()はターンの最終メッセージを示します。
      if event.is_final_response():
         if event.content and event.content.parts:
            # 最初のパートにテキスト応答があると仮定
            final_response_text = event.content.parts[0].text
         elif event.actions and event.actions.escalate: # エラー/エスカレーションの可能性を処理
            final_response_text = f"エージェントがエスカレーション: {event.error_message or '特定のメッセージなし。'}"
            # 必要に応じてここに他のチェックを追加（例：特定のエラーコードなど）
         break # 最終応答が見つかったらイベント処理を停止

   print(f"<<< エージェント応答: {final_response_text}")

 ---
 
 **5\. 会話を実行する**
 
 最後に、エージェントにいくつかのクエリを送信して、セットアップをテストしましょう。メイン`async`関数内で`async`呼び出しをラップし、await`を使用して実行します。
 
 出力を観察してください：
 
 * ユーザークエリを確認します。
 * エージェントがツールを使用するときに`--- ツール: get_weather が...のために呼び出されました ---`ログに注目してください。
 * エージェントの最終応答を観察してください。天気データが利用できない場合（パリの場合）の処理方法も含まれています。

In [17]:
# @title 初期会話の実行

# 対話ヘルパーを待機するための非同期関数が必要です
async def run_conversation():
    await call_agent_async("ロンドンの天気はどうですか？",
                                        runner=runner,
                                        user_id=USER_ID,
                                        session_id=SESSION_ID)

    await call_agent_async("パリはどうですか？",
                                        runner=runner,
                                        user_id=USER_ID,
                                        session_id=SESSION_ID) # ツールのエラーメッセージを期待

    await call_agent_async("ニューヨークの天気を教えてください",
                                        runner=runner,
                                        user_id=USER_ID,
                                        session_id=SESSION_ID)

# 非同期コンテキスト（ColabやJupyterなど）でawaitを使用して会話を実行
await run_conversation()




>>> ユーザークエリ: ロンドンの天気はどうですか？


--- ツール: get_weather が都市: London のために呼び出されました ---
<<< エージェント応答: ロンドンの天気は曇りで、気温は15℃です。

>>> ユーザークエリ: パリはどうですか？
--- ツール: get_weather が都市: Paris のために呼び出されました ---
<<< エージェント応答: 申し訳ありません。パリの天気情報を取得できませんでした。

>>> ユーザークエリ: ニューヨークの天気を教えてください
--- ツール: get_weather が都市: New York のために呼び出されました ---
<<< エージェント応答: ニューヨークの天気は晴れで、気温は25℃です。


 ---
 
 おめでとうございます！最初のADKエージェントの構築と対話に成功しました。エージェントはユーザーのリクエストを理解し、ツールを使用し情報を検索し、ツールの結果に基づいて適切に応答します。
 
 次のステップでは、このエージェントを動かす基盤となる言語モデルを簡単に切り替える方法を探ります。

 ## ステップ2: LiteLLMを使用したマルチモデル対応
 ステップ1では、特定のGeminiモデルを利用した機能的な天気エージェントを構築しました。これは効果的ですが、実際のアプリケーションでは*異なる*大規模言語モデル（LLM）を使用する柔軟性が役立つことがよくあります。なぜでしょうか？
 * **パフォーマンス:** 一部のモデルは特定のタスク（例：コーディング、推論、創造的な文章作成）に優れています。
 * **コスト:** モデルによって価格設定が異なります。
 * **機能:** モデルは多様な機能、コンテキストウィンドウサイズ、微調整オプションを提供します。
 * **可用性/冗長性:** 代替手段を持つことで、あるプロバイダーに問題が発生しても、アプリケーションが機能し続けることを保証します。
 ADKは[**LiteLLM**](https://github.com/BerriAI/litellm)ライブラリとの統合を通じて、モデル間の切り替えをシームレスにします。LiteLLMは100以上の異なるLLMへの一貫したインターフェースとして機能します。
 **このステップでは以下を行います：**
 1. `LiteLlm`ラッパーを使用して、OpenAI（GPT）やAnthropic（Claude）などのプロバイダーからモデルを使用するようにADK `Agent`を設定する方法を学びます。
 2. 天気エージェントのインスタンスを定義、設定（それぞれ独自のセッションとランナーを持つ）し、それぞれ異なるLLMによってサポートされたものをすぐにテストします。
 3. これらの異なるエージェントと対話して、同じ基本ツールを使用していても、応答の潜在的な違いを観察します。

---

**1\. Import `LiteLlm`**

We imported this during the initial setup (Step 0), but it's the key component for multi-model support:

In [18]:
# @title 1. Import LiteLlm
from google.adk.models.lite_llm import LiteLlm

 **2\. マルチモデルエージェントの定義とテスト**
 
 モデル名の文字列だけを渡す（デフォルトではGoogleのGeminiモデルが使用される）代わりに、希望するモデル識別子文字列を`LiteLlm`クラス内でラップします。
 
 *   **重要な概念：`LiteLlm`ラッパー：** `LiteLlm(model="provider/model_name")`構文は、このエージェントへのリクエストを指定されたモデルプロバイダーにLiteLLMライブラリを通じてルーティングするようADKに指示します。
 
 ステップ0で、OpenAIとAnthropicに必要なAPIキーを設定したことを確認してください。セットアップ直後に各エージェントと対話するために、`call_agent_async`関数（以前に定義され、現在は`runner`、`user_id`、`session_id`を受け入れる）を使用します。
 
 以下の各ブロックでは：
 *   特定のLiteLLMモデル（`MODEL_GPT_o3_MINI`または`MODEL_CLAUDE_SONNET`）を使用してエージェントを定義します。
 *   そのエージェントのテスト実行専用の*新しい、個別の*`InMemorySessionService`とセッションを作成します。これにより、このデモンストレーションの会話履歴が分離されます。
 *   特定のエージェントとそのセッションサービス用に設定された`Runner`を作成します。
 *   すぐに`call_agent_async`を呼び出してクエリを送信し、エージェントをテストします。
 
 **ベストプラクティス：** モデル名には定数（ステップ0で定義された`MODEL_GPT_o3_MINI`、`MODEL_CLAUDE_SONNET`など）を使用して、タイプミスを防ぎ、コードを管理しやすくします。
 
 **エラー処理：** エージェント定義を`try...except`ブロックでラップします。これにより、特定のプロバイダーのAPIキーが欠落しているか無効な場合でも、コードセル全体が失敗するのを防ぎ、設定されている*モデル*でチュートリアルを続行できます。
 
 まず、OpenAIのGPT-4oを使用してエージェントを作成しテストしましょう。

In [19]:
# @title GPTエージェントの定義とテスト

# ステップ1で定義した'get_weather'関数が環境内に定義されていることを確認してください。
# 以前に定義した'call_agent_async'が定義されていることを確認してください。

# --- GPT-4oを使用したエージェント ---
weather_agent_gpt = None # Noneで初期化
runner_gpt = None      # ランナーをNoneで初期化

try:
    weather_agent_gpt = Agent(
        name="weather_agent_gpt",
        # 重要な変更: LiteLLMモデル識別子をラップする
        model=LiteLlm(model=MODEL_GPT_o3_MINI),
        description="天気情報を提供します（GPT-4oを使用）。",
        instruction="あなたはGPT-4oを搭載した役立つ天気アシスタントです。"
                    "都市の天気リクエストには'get_weather'ツールを使用してください。"
                    "ツールの出力ステータスに基づいて、成功したレポートや丁寧なエラーメッセージを明確に提示してください。",
        tools=[get_weather], # 同じツールを再利用
    )
    print(f"エージェント '{weather_agent_gpt.name}' がモデル '{MODEL_GPT_o3_MINI}' を使用して作成されました。")

    # InMemorySessionServiceは、このチュートリアル用のシンプルな非永続的ストレージです。
    session_service_gpt = InMemorySessionService() # 専用のサービスを作成

    # インタラクションコンテキストを識別するための定数を定義
    APP_NAME_GPT = "weather_tutorial_app_gpt" # このテスト用のユニークなアプリ名
    USER_ID_GPT = "user_1_gpt"
    SESSION_ID_GPT = "session_001_gpt" # シンプルにするために固定IDを使用

    # 会話が行われる特定のセッションを作成
    session_gpt = session_service_gpt.create_session(
        app_name=APP_NAME_GPT,
        user_id=USER_ID_GPT,
        session_id=SESSION_ID_GPT
    )
    print(f"セッションが作成されました: アプリ='{APP_NAME_GPT}', ユーザー='{USER_ID_GPT}', セッション='{SESSION_ID_GPT}'")

    # このエージェントとそのセッションサービス専用のランナーを作成
    runner_gpt = Runner(
        agent=weather_agent_gpt,
        app_name=APP_NAME_GPT,       # 特定のアプリ名を使用
        session_service=session_service_gpt # 特定のセッションサービスを使用
        )
    print(f"エージェント '{runner_gpt.agent.name}' 用のランナーが作成されました。")

    # --- GPTエージェントのテスト ---
    print("\n--- GPTエージェントのテスト ---")
    # call_agent_asyncが正しいランナー、user_id、session_idを使用していることを確認
    await call_agent_async(query = "東京の天気はどうですか？",
                            runner=runner_gpt,
                            user_id=USER_ID_GPT,
                            session_id=SESSION_ID_GPT)

except Exception as e:
    print(f"❌ GPTエージェント '{MODEL_GPT_o3_MINI}' の作成または実行ができませんでした。APIキーとモデル名を確認してください。エラー: {e}")


エージェント 'weather_agent_gpt' がモデル 'openai/o3-mini-2025-01-31' を使用して作成されました。
セッションが作成されました: アプリ='weather_tutorial_app_gpt', ユーザー='user_1_gpt', セッション='session_001_gpt'
エージェント 'weather_agent_gpt' 用のランナーが作成されました。

--- GPTエージェントのテスト ---

>>> ユーザークエリ: 東京の天気はどうですか？


--- ツール: get_weather が都市: Tokyo のために呼び出されました ---
<<< エージェント応答: 東京は現在、軽い雨が降っており、気温は約18°Cです。


Next, we'll do the same for Anthropic's Claude Sonnet.

In [ ]:
# @title Define and Test Claude Agent

# Make sure 'get_weather' function from Step 1 is defined in your environment.
# Make sure 'call_agent_async' is defined from earlier.

# --- Agent using Claude Sonnet ---
weather_agent_claude = None # Initialize to None
runner_claude = None      # Initialize runner to None

try:
    weather_agent_claude = Agent(
        name="weather_agent_claude",
        # Key change: Wrap the LiteLLM model identifier
        model=LiteLlm(model=MODEL_CLAUDE_SONNET),
        description="Provides weather information (using Claude Sonnet).",
        instruction="You are a helpful weather assistant powered by Claude Sonnet. "
                    "Use the 'get_weather' tool for city weather requests. "
                    "Analyze the tool's dictionary output ('status', 'report'/'error_message'). "
                    "Clearly present successful reports or polite error messages.",
        tools=[get_weather], # Re-use the same tool
    )
    print(f"Agent '{weather_agent_claude.name}' created using model '{MODEL_CLAUDE_SONNET}'.")

    # InMemorySessionService is simple, non-persistent storage for this tutorial.
    session_service_claude = InMemorySessionService() # Create a dedicated service

    # Define constants for identifying the interaction context
    APP_NAME_CLAUDE = "weather_tutorial_app_claude" # Unique app name
    USER_ID_CLAUDE = "user_1_claude"
    SESSION_ID_CLAUDE = "session_001_claude" # Using a fixed ID for simplicity

    # Create the specific session where the conversation will happen
    session_claude = session_service_claude.create_session(
        app_name=APP_NAME_CLAUDE,
        user_id=USER_ID_CLAUDE,
        session_id=SESSION_ID_CLAUDE
    )
    print(f"Session created: App='{APP_NAME_CLAUDE}', User='{USER_ID_CLAUDE}', Session='{SESSION_ID_CLAUDE}'")

    # Create a runner specific to this agent and its session service
    runner_claude = Runner(
        agent=weather_agent_claude,
        app_name=APP_NAME_CLAUDE,       # Use the specific app name
        session_service=session_service_claude # Use the specific session service
        )
    print(f"Runner created for agent '{runner_claude.agent.name}'.")

    # --- Test the Claude Agent ---
    print("\n--- Testing Claude Agent ---")
    # Ensure call_agent_async uses the correct runner, user_id, session_id
    await call_agent_async(query = "Weather in London please.",
                           runner=runner_claude,
                           user_id=USER_ID_CLAUDE,
                           session_id=SESSION_ID_CLAUDE)

except Exception as e:
    print(f"❌ Could not create or run Claude agent '{MODEL_CLAUDE_SONNET}'. Check API Key and model name. Error: {e}")

 両方のコードブロックの出力を注意深く観察してください。以下のことが確認できるはずです：
 
 1.  各エージェント（`weather_agent_gpt`、`weather_agent_claude`）は（APIキーが有効であれば）正常に作成されます。
 2.  各エージェント専用のセッションとランナーが設定されています。
 3.  各エージェントはクエリを処理する際に`get_weather`ツールを使用する必要性を正しく識別します（`--- Tool: get_weather called... ---`というログが表示されます）。
 4.  *基盤となるツールのロジック*は同一のままで、常に私たちのモックデータを返します。
 5.  ただし、各エージェントが生成する**最終的なテキスト応答**は、表現、トーン、フォーマットにわずかな違いがある場合があります。これは、指示プロンプトが異なるLLM（GPT-4oとClaude Sonnet）によって解釈され実行されるためです。
 
 このステップは、ADK + LiteLLMが提供するパワーと柔軟性を示しています。コアアプリケーションロジック（ツール、基本的なエージェント構造）を一貫させながら、さまざまなLLMを使用してエージェントを簡単に実験し、デプロイすることができます。
 
 次のステップでは、単一のエージェントを超えて、エージェントが互いにタスクを委任できる小さなチームを構築します！
 
 ---

 ## ステップ3: エージェントチームの構築 \- 挨拶と別れの委任
 ステップ1と2では、天気検索のみに焦点を当てた単一のエージェントを構築し実験しました。特定のタスクには効果的ですが、実際のアプリケーションではより多様なユーザーとのやり取りを処理する必要があります。単一の天気エージェントにさらにツールや複雑な指示を追加することも*できますが*、これはすぐに管理が難しくなり、効率が低下する可能性があります。
 より堅牢なアプローチは**エージェントチーム**を構築することです。これには以下が含まれます：
 1. 複数の**専門エージェント**を作成し、それぞれが特定の機能（例：天気用、挨拶用、計算用）のために設計されています。
 2. 最初のユーザーリクエストを受け取る**ルートエージェント**（またはオーケストレーター）を指定します。
 3. ルートエージェントがユーザーの意図に基づいて、最も適切な専門サブエージェントにリクエストを**委任**できるようにします。
 **なぜエージェントチームを構築するのか？**
 * **モジュール性：** 個々のエージェントの開発、テスト、保守が容易になります。
 * **専門化：** 各エージェントは特定のタスクに合わせて微調整（指示、モデル選択）できます。
 * **拡張性：** 新しいエージェントを追加することで、新しい機能を簡単に追加できます。
 * **効率性：** より単純なタスク（挨拶など）には、潜在的により単純/安価なモデルを使用できます。
 **このステップでは以下を行います：**
 1. 挨拶（`say_hello`）と別れ（`say_goodbye`）を処理するための単純なツールを定義します。
 2. `greeting_agent`と`farewell_agent`という2つの新しい専門サブエージェントを作成します。
 3. メインの天気エージェント（`weather_agent_v2`）を**ルートエージェント**として機能するように更新します。
 4. ルートエージェントをサブエージェントで構成し、**自動委任**を可能にします。
 5. ルートエージェントにさまざまなタイプのリクエストを送信して、委任フローをテストします。

---

**1\. Define Tools for Sub-Agents**

First, let's create the simple Python functions that will serve as tools for our new specialist agents. Remember, clear docstrings are vital for the agents that will use them.

In [18]:
# @title 挨拶と別れのエージェント用ツールを定義する

# ステップ1の'get_weather'が独立して実行する場合に利用可能であることを確認してください。
# def get_weather(city: str) -> dict: ... (ステップ1から)

def say_hello(name: str = "there") -> str:
    """ユーザーに簡単な挨拶を提供し、オプションで名前で呼びかけます。

    Args:
        name (str, optional): 挨拶する相手の名前。デフォルトは "there"。

    Returns:
        str: 友好的な挨拶メッセージ。
    """
    print(f"--- ツール: say_hello が name: {name} で呼び出されました ---")
    return f"こんにちは、{name}さん！"

def say_goodbye() -> str:
    """会話を締めくくるための簡単な別れのメッセージを提供します。"""
    print(f"--- ツール: say_goodbye が呼び出されました ---")
    return "さようなら！良い一日を。"

print("挨拶と別れのツールが定義されました。")

# オプションの自己テスト
print(say_hello("アリス"))
print(say_goodbye())

挨拶と別れのツールが定義されました。
--- ツール: say_hello が name: アリス で呼び出されました ---
こんにちは、アリスさん！
--- ツール: say_goodbye が呼び出されました ---
さようなら！良い一日を。


 ---
 
 **2\. サブエージェントの定義（挨拶と別れ）**
 
 ここで、専門家のための`Agent`インスタンスを作成します。彼らの非常に焦点を絞った`instruction`と、重要なことに、明確な`description`に注目してください。`description`は*ルートエージェント*が*いつ*これらのサブエージェントに委任するかを決定するための主要な情報です。
 
 これらのサブエージェントに異なるLLMを使用することもできます！挨拶エージェントにはGPT-4oを割り当て、別れエージェントもGPT-4oを使用したままにしましょう（必要に応じてAPIキーが設定されていれば、一方をClaudeやGeminiに簡単に切り替えることができます）。
 
 **ベストプラクティス:** サブエージェントの`description`フィールドは、その特定の能力を正確かつ簡潔に要約する必要があります。これは効果的な自動委任のために非常に重要です。
 
 **ベストプラクティス:** サブエージェントの`instruction`フィールドは、その限られた範囲に合わせて調整し、何をすべきか、そして何をすべきでないか（例：「あなたの*唯一の*タスクは...」）を正確に伝える必要があります。

In [19]:
# @title 挨拶と別れのサブエージェントを定義する

# LiteLlmがインポートされ、APIキーが設定されていることを確認（ステップ0/2から）
# from google.adk.models.lite_llm import LiteLlm
# MODEL_GPT_o3_MINI, MODEL_CLAUDE_SONNET などが定義されているはず

# --- 挨拶エージェント ---
greeting_agent = None
try:
    greeting_agent = Agent(
        # シンプルなタスクには、より安価な可能性のあるモデルを使用
        model=LiteLlm(model=MODEL_GPT_o3_MINI),
        name="greeting_agent",
        instruction="あなたは挨拶エージェントです。あなたの唯一のタスクはユーザーに友好的な挨拶を提供することです。"
                    "'say_hello'ツールを使用して挨拶を生成してください。"
                    "ユーザーが名前を提供した場合は、必ずそれをツールに渡してください。"
                    "他の会話やタスクには関与しないでください。",
        description="'say_hello'ツールを使用してシンプルな挨拶を処理します。", # 委任に重要
        tools=[say_hello],
    )
    print(f"✅ エージェント '{greeting_agent.name}' がモデル '{MODEL_GPT_o3_MINI}' を使用して作成されました。")
except Exception as e:
    print(f"❌ 挨拶エージェントを作成できませんでした。APIキー ({MODEL_GPT_o3_MINI}) を確認してください。エラー: {e}")

# --- 別れエージェント ---
farewell_agent = None
try:
    farewell_agent = Agent(
        # 同じまたは異なるモデルを使用可能
        model=LiteLlm(model=MODEL_GPT_o3_MINI), # この例ではGPTを使用
        name="farewell_agent",
        instruction="あなたは別れエージェントです。あなたの唯一のタスクは丁寧な別れのメッセージを提供することです。"
                    "ユーザーが会話を終了する意思を示した場合（例：「さようなら」、「バイバイ」、「ありがとう、さようなら」、「また会いましょう」などの言葉を使用）、"
                    "'say_goodbye'ツールを使用してください。"
                    "他のアクションは実行しないでください。",
        description="'say_goodbye'ツールを使用してシンプルな別れの挨拶を処理します。", # 委任に重要
        tools=[say_goodbye],
    )
    print(f"✅ エージェント '{farewell_agent.name}' がモデル '{MODEL_GPT_o3_MINI}' を使用して作成されました。")
except Exception as e:
    print(f"❌ 別れエージェントを作成できませんでした。APIキー ({MODEL_GPT_o3_MINI}) を確認してください。エラー: {e}")

✅ エージェント 'greeting_agent' がモデル 'openai/o3-mini-2025-01-31' を使用して作成されました。
✅ エージェント 'farewell_agent' がモデル 'openai/o3-mini-2025-01-31' を使用して作成されました。


 ---
 
 **3\. ルートエージェント（Weather Agent v2）とサブエージェントの定義**
 
 ここで、`weather_agent`をアップグレードします。主な変更点は以下の通りです：
 
 * `sub_agents`パラメータの追加：先ほど作成した`greeting_agent`と`farewell_agent`のインスタンスを含むリストを渡します。
 * `instruction`の更新：ルートエージェントに対して、そのサブエージェントについて*明示的に*伝え、*いつ*タスクを委任すべきかを指示します。
 
 **重要概念：自動委任（Auto Flow）** `sub_agents`リストを提供することで、ADKは自動委任を可能にします。ルートエージェントがユーザークエリを受け取ると、そのLLMは自身の指示とツールだけでなく、各サブエージェントの`description`も考慮します。LLMがクエリがサブエージェントの説明された能力（例：「シンプルな挨拶を処理する」）により適合すると判断した場合、そのターンで*制御を移す*ための特別な内部アクションを自動的に生成します。サブエージェントはその後、独自のモデル、指示、ツールを使用してクエリを処理します。
 
 **ベストプラクティス：** ルートエージェントの指示が委任の決定を明確に導くようにしてください。サブエージェントを名前で言及し、委任が発生すべき条件を説明してください。

In [20]:
# @title ルートエージェントとサブエージェントの定義

# サブエージェントが正常に作成されていることと、元の'get_weather'ツールが定義されていることを確認します。
root_agent = None
runner_root = None # ランナーを初期化

if greeting_agent and farewell_agent and 'get_weather' in globals():
    # オーケストレーションを処理するために高性能なO3モデルを使用しましょう
    root_agent_model = LiteLlm(model=MODEL_GPT_o3_MINI)

    weather_agent_team = Agent(
        name="weather_agent_v2", # 新しいバージョン名を付ける
        model=root_agent_model,
        description="メインコーディネーターエージェント。天気リクエストを処理し、挨拶/別れを専門家に委任します。",
        instruction="あなたはチームを調整するメイン天気エージェントです。あなたの主な責任は天気情報を提供することです。"
                    "特定の天気リクエスト（例：「ロンドンの天気」）に対してのみ'get_weather'ツールを使用してください。"
                    "あなたには専門のサブエージェントがいます："
                    "1. 'greeting_agent'：「こんにちは」、「やあ」などの簡単な挨拶を処理します。これらの場合は委任してください。"
                    "2. 'farewell_agent'：「さようなら」、「また会いましょう」などの簡単な別れを処理します。これらの場合は委任してください。"
                    "ユーザーのクエリを分析してください。挨拶の場合は'greeting_agent'に委任し、別れの場合は'farewell_agent'に委任してください。"
                    "天気のリクエストの場合は、'get_weather'を使用して自分自身で処理してください。"
                    "それ以外の場合は、適切に応答するか、対応できないことを述べてください。",
        tools=[get_weather], # ルートエージェントは主要タスクのために天気ツールが必要
        # 重要な変更：ここでサブエージェントをリンク！
        sub_agents=[greeting_agent, farewell_agent]
    )
    print(f"✅ ルートエージェント '{weather_agent_team.name}' がモデル '{root_agent_model}' を使用して作成されました。サブエージェント: {[sa.name for sa in weather_agent_team.sub_agents]}")

else:
    print("❌ 1つ以上のサブエージェントが初期化に失敗したか、'get_weather'ツールが見つからないため、ルートエージェントを作成できません。")
    if not greeting_agent: print(" - 挨拶エージェントが見つかりません。")
    if not farewell_agent: print(" - 別れエージェントが見つかりません。")
    if 'get_weather' not in globals(): print(" - get_weather関数が見つかりません。")


✅ ルートエージェント 'weather_agent_v2' がモデル 'model='openai/o3-mini-2025-01-31' llm_client=<google.adk.models.lite_llm.LiteLLMClient object at 0x7f9afa82f740>' を使用して作成されました。サブエージェント: ['greeting_agent', 'farewell_agent']


 ---
 
 **4\. エージェントチームとの対話**
 
 ルートエージェント（`weather_agent_team` - *注意：この変数名が前のコードブロックで定義されたものと一致していることを確認してください。おそらく `# @title ルートエージェントとサブエージェントの定義` で `root_agent` という名前になっているかもしれません*）を専門のサブエージェントと共に定義したので、委任メカニズムをテストしましょう。
 
 次のコードブロックでは：
 
 1.  `async`関数 `run_team_conversation` を定義します。
 2.  この関数内で、このテスト実行専用の*新しい専用* `InMemorySessionService` と特定のセッション（`session_001_agent_team`）を作成します。これにより、チームダイナミクスをテストするための会話履歴が分離されます。
 3.  `Runner`（`runner_agent_team`）を作成し、`weather_agent_team`（ルートエージェント）と専用セッションサービスを使用するように構成します。
 4.  更新された `call_agent_async` 関数を使用して、異なるタイプのクエリ（挨拶、天気リクエスト、別れ）を `runner_agent_team` に送信します。このテスト専用にランナー、ユーザーID、セッションIDを明示的に渡します。
 5.  `run_team_conversation` 関数をすぐに実行します。
 
 予想されるフローは次のとおりです：
 
 1.  「こんにちは！」というクエリが `runner_agent_team` に送られます。
 2.  ルートエージェント（`weather_agent_team`）がそれを受け取り、その指示と `greeting_agent` の説明に基づいて、タスクを委任します。
 3.  `greeting_agent` がクエリを処理し、`say_hello` ツールを呼び出して、応答を生成します。
 4.  「ニューヨークの天気は？」というクエリは委任*されず*、ルートエージェントが直接 `get_weather` ツールを使用して処理します。
 5.  「ありがとう、さようなら！」というクエリは `farewell_agent` に委任され、`say_goodbye` ツールを使用します。
 


In [23]:
# @title エージェントチームとの対話

# ルートエージェント（前のセルの 'weather_agent_team' または 'root_agent'）が定義されていることを確認します。
# call_agent_async 関数が定義されていることを確認します。

# ルートエージェント変数が存在するか確認してから会話関数を定義します
root_agent_var_name = 'root_agent' # ステップ3ガイドからのデフォルト名
if 'weather_agent_team' in globals(): # ユーザーが代わりにこの名前を使用したか確認
    root_agent_var_name = 'weather_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ ルートエージェント（'root_agent'または'weather_agent_team'）が見つかりません。run_team_conversationを定義できません。")
    # コードブロックが実行された場合にNameErrorを防ぐためのダミー値を割り当て
    root_agent = None

if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    async def run_team_conversation():
        print("\n--- エージェントチーム委任のテスト ---")
        # InMemorySessionServiceは、このチュートリアル用のシンプルな非永続的ストレージです。
        session_service = InMemorySessionService()

        # インタラクションコンテキストを識別するための定数を定義
        APP_NAME = "weather_tutorial_agent_team"
        USER_ID = "user_1_agent_team"
        SESSION_ID = "session_001_agent_team" # 簡略化のために固定IDを使用

        # 会話が行われる特定のセッションを作成
        session = session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID
        )
        print(f"セッション作成: アプリ='{APP_NAME}', ユーザー='{USER_ID}', セッション='{SESSION_ID}'")

        # --- 実際のルートエージェントオブジェクトを取得 ---
        # 決定された変数名を使用
        actual_root_agent = globals()[root_agent_var_name]

        # このエージェントチームテスト専用のランナーを作成
        runner_agent_team = Runner(
            agent=actual_root_agent, # ルートエージェントオブジェクトを使用
            app_name=APP_NAME,       # 特定のアプリ名を使用
            session_service=session_service # 特定のセッションサービスを使用
            )
        # 実際のルートエージェントの名前を表示するための修正されたprint文
        print(f"エージェント '{actual_root_agent.name}' のランナーが作成されました。")

        # 常に正しいIDを渡してルートエージェントのランナーを介して対話
        await call_agent_async(query = "こんにちは！",
                                runner=runner_agent_team,
                                user_id=USER_ID,
                                session_id=SESSION_ID)
        await call_agent_async(query = "ニューヨークの天気は？",
                                runner=runner_agent_team,
                                user_id=USER_ID,
                                session_id=SESSION_ID)
        await call_agent_async(query = "ありがとう、さようなら！",
                                runner=runner_agent_team,
                                user_id=USER_ID,
                                session_id=SESSION_ID)

    # 会話を実行
    # 注意: ルートおよびサブエージェントで使用されるモデルのAPIキーが必要かもしれません！
    await run_team_conversation()
else:
    print("\n⚠️ 前のステップでルートエージェントが正常に定義されなかったため、エージェントチーム会話をスキップします。")



--- エージェントチーム委任のテスト ---
セッション作成: アプリ='weather_tutorial_agent_team', ユーザー='user_1_agent_team', セッション='session_001_agent_team'
エージェント 'weather_agent_v2' のランナーが作成されました。

>>> ユーザークエリ: こんにちは！


--- ツール: say_hello が name:  で呼び出されました ---
<<< エージェント応答: こんにちは、さん！

>>> ユーザークエリ: ニューヨークの天気は？
--- ツール: get_weather が都市: ニューヨーク のために呼び出されました ---
<<< エージェント応答: 申し訳ありませんが、ニューヨークの天気情報が見つかりませんでした。

>>> ユーザークエリ: ありがとう、さようなら！
--- ツール: say_goodbye が呼び出されました ---
--- ツール: say_goodbye が呼び出されました ---
<<< エージェント応答: さようなら！またお会いできる日を楽しみにしています。


 ---
 
 出力ログをよく見てください。特に `--- Tool: ... called ---` メッセージに注目してください。以下のことが確認できるはずです：
 
 *   「こんにちは！」に対して、`say_hello` ツールが呼び出されました（`greeting_agent` が処理したことを示しています）。
 *   「ニューヨークの天気は？」に対して、`get_weather` ツールが呼び出されました（ルートエージェントが処理したことを示しています）。
 *   「ありがとう、さようなら！」に対して、`say_goodbye` ツールが呼び出されました（`farewell_agent` が処理したことを示しています）。
 
 これにより、**自動委任**が成功したことが確認できます！ルートエージェントは、その指示と `sub_agents` の `description` に導かれて、ユーザーのリクエストをチーム内の適切な専門エージェントに正しく振り分けました。
 
 これで、複数のエージェントが連携するアプリケーション構造を作成できました。このモジュラー設計は、より複雑で高機能なエージェントシステムを構築するための基本です。次のステップでは、セッション状態を使用してターン間で情報を記憶する能力をエージェントに与えます。

 ## ステップ4: セッション状態を使用したメモリと個人化の追加
 
 これまで、エージェントチームは委任を通じて異なるタスクを処理できましたが、各対話は新しく始まります - エージェントはセッション内の過去の会話やユーザー設定を記憶していません。より洗練されたコンテキスト対応の体験を作成するために、エージェントには**メモリ**が必要です。ADKはこれを**セッション状態**を通じて提供します。
 
 **セッション状態とは？**
 
 * 特定のユーザーセッション（`APP_NAME`、`USER_ID`、`SESSION_ID`で識別）に紐づけられたPythonディクショナリ（`session.state`）です。
 * そのセッション内の*複数の会話ターンにわたって*情報を保持します。
 * エージェントとツールはこの状態から読み取りや書き込みができ、詳細を記憶し、動作を適応させ、応答をパーソナライズすることができます。
 
 **エージェントが状態とやり取りする方法：**
 
 1. **`ToolContext`（主要な方法）:** ツールは`ToolContext`オブジェクト（最後の引数として宣言されている場合、ADKによって自動的に提供される）を受け入れることができます。このオブジェクトは`tool_context.state`を通じてセッション状態への直接アクセスを提供し、ツールが実行*中に*設定を読み取ったり結果を保存したりできるようにします。
 2. **`output_key`（エージェント応答の自動保存）:** `Agent`は`output_key="your_key"`で構成できます。ADKはその後、ターンごとのエージェントの最終的なテキスト応答を`session.state["your_key"]`に自動的に保存します。
 
 **このステップでは、天気ボットチームを以下のように強化します：**
 
 1. 状態を分離して示すために**新しい**`InMemorySessionService`を使用します。
 2. `temperature_unit`のユーザー設定でセッション状態を初期化します。
 3. `ToolContext`を通じてこの設定を読み取り、出力形式（摂氏/華氏）を調整する状態対応バージョンの天気ツール（`get_weather_stateful`）を作成します。
 4. ルートエージェントをこの状態対応ツールを使用するように更新し、`output_key`で構成して、最終的な天気レポートをセッション状態に自動的に保存するようにします。
 5. 会話を実行して、初期状態がツールにどのように影響するか、手動での状態変更がその後の動作をどのように変更するか、そして`output_key`がエージェントの応答をどのように保持するかを観察します。

 ---
 
 **1\. 新しいセッションサービスと状態の初期化**
 
 前のステップからの干渉なしに状態管理を明確に示すために、新しい`InMemorySessionService`をインスタンス化します。また、ユーザーの好みの温度単位を定義する初期状態でセッションを作成します。

In [24]:
# @title 1. 新しいセッションサービスと状態の初期化

# 必要なセッションコンポーネントをインポート
from google.adk.sessions import InMemorySessionService

# この状態デモンストレーション用に新しいセッションサービスインスタンスを作成
session_service_stateful = InMemorySessionService()
print("✅ 状態デモンストレーション用の新しいInMemorySessionServiceが作成されました。")

# このチュートリアルのパート用に新しいセッションIDを定義
SESSION_ID_STATEFUL = "session_state_demo_001"
USER_ID_STATEFUL = "user_state_demo"

# 初期状態データを定義 - ユーザーは最初に摂氏を好む
# Define initial state data - user prefers Celsius initially
initial_state = {
    "user_preference_temperature_unit": "Celsius"
}

# 初期状態を提供してセッションを作成
session_stateful = session_service_stateful.create_session(
    app_name=APP_NAME, # 一貫したアプリ名を使用
    user_id=USER_ID_STATEFUL,
    session_id=SESSION_ID_STATEFUL,
    state=initial_state # <<< 作成中に状態を初期化
)
print(f"✅ ユーザー '{USER_ID_STATEFUL}' のセッション '{SESSION_ID_STATEFUL}' が作成されました。")

# 初期状態が正しく設定されたことを確認
retrieved_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                            user_id=USER_ID_STATEFUL,
                                                            session_id = SESSION_ID_STATEFUL)
print("\n--- 初期セッション状態 ---")
if retrieved_session:
    print(retrieved_session.state)
else:
    print("エラー: セッションを取得できませんでした。")

✅ 状態デモンストレーション用の新しいInMemorySessionServiceが作成されました。
✅ ユーザー 'user_state_demo' のセッション 'session_state_demo_001' が作成されました。

--- 初期セッション状態 ---
{'user_preference_temperature_unit': 'Celsius'}


 ---
 
 **2\. 状態を認識する天気ツールの作成 (`get_weather_stateful`)**
 
 ここでは、天気ツールの新しいバージョンを作成します。その主な特徴は、`tool_context: ToolContext`を受け入れることで、`tool_context.state`にアクセスできることです。これにより、`user_preference_temperature_unit`を読み取り、それに応じて温度をフォーマットします。
 
 
 * **重要な概念: `ToolContext`** このオブジェクトは、ツールロジックがセッションのコンテキストと対話するための橋渡しとなり、状態変数の読み書きが可能になります。ADKは、ツール関数の最後のパラメータとして定義されていれば、自動的にこれを注入します。
 
 
 * **ベストプラクティス:** 状態から読み取る際は、`dictionary.get('key', default_value)`を使用して、キーがまだ存在しない場合に対応し、ツールがクラッシュしないようにします。

In [26]:
from google.adk.tools.tool_context import ToolContext

def get_weather_stateful(city: str, tool_context: ToolContext) -> dict:
    """セッション状態に基づいて天気を取得し、温度単位を変換します。"""
    print(f"--- ツール: get_weather_stateful が {city} のために呼び出されました ---")

    # --- 状態から設定を読み取る ---
    preferred_unit = tool_context.state.get("user_preference_temperature_unit", "Celsius") # デフォルトは摂氏
    print(f"--- ツール: 状態 'user_preference_temperature_unit' を読み取り中: {preferred_unit} ---")

    city_normalized = city.lower().replace(" ", "")

    # モック天気データ（内部では常に摂氏で保存）
    mock_weather_db = {
        "newyork": {"temp_c": 25, "condition": "sunny"},
        "london": {"temp_c": 15, "condition": "cloudy"},
        "tokyo": {"temp_c": 18, "condition": "light rain"},
        "東京": {"temp_c": 54, "condition": "晴れ"},
    }

    if city_normalized in mock_weather_db:
        data = mock_weather_db[city_normalized]
        temp_c = data["temp_c"]
        condition = data["condition"]

        # 状態の設定に基づいて温度をフォーマット
        if preferred_unit == "Fahrenheit":
            temp_value = (temp_c * 9/5) + 32 # 華氏に変換
            temp_unit = "°F"
        else: # デフォルトは摂氏
            temp_value = temp_c
            temp_unit = "°C"

        report = f"{city.capitalize()}の天気は{condition}で、気温は{temp_value:.0f}{temp_unit}です。"
        result = {"status": "success", "report": report}
        print(f"--- ツール: {preferred_unit}で天気レポートを生成しました。結果: {result} ---")

        # 状態に書き戻す例（このツールではオプション）
        tool_context.state["last_city_checked_stateful"] = city
        print(f"--- ツール: 状態 'last_city_checked_stateful' を更新: {city} ---")

        return result
    else:
        # 都市が見つからない場合の処理
        error_msg = f"申し訳ありませんが、'{city}'の天気情報はありません。"
        print(f"--- ツール: 都市 '{city}' が見つかりません。 ---")
        return {"status": "error", "error_message": error_msg}

print("✅ 状態を認識する 'get_weather_stateful' ツールが定義されました。")


✅ 状態を認識する 'get_weather_stateful' ツールが定義されました。


 ---
 
 **3\. サブエージェントの再定義とルートエージェントの更新**
 
 このステップが自己完結型で正しく構築されるように、まずステップ3と同様に`greeting_agent`と`farewell_agent`を再定義します。次に、新しいルートエージェント（`weather_agent_v4_stateful`）を定義します：
 
 * 新しい`get_weather_stateful`ツールを使用します。
 * 委任のためのあいさつと別れのサブエージェントを含みます。
 * **重要なポイント**として、`output_key="last_weather_report"`を設定し、最終的な天気レポートをセッション状態に自動的に保存します。

In [27]:
# @title 3. サブエージェントの再定義とルートエージェントの更新（output_key付き）

# 必要なインポート: Agent, LiteLlm, Runner
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner
# ツール 'say_hello', 'say_goodbye' がステップ3から定義されていることを確認
# モデル定数 MODEL_GPT_o3_MINI, MODEL_GEMINI_2_5_PRO などが定義されていることを確認

# --- あいさつエージェントの再定義（ステップ3から） ---
greeting_agent = None
try:
    greeting_agent = Agent(
        model=LiteLlm(model=MODEL_GPT_o3_MINI),
        name="greeting_agent",
        instruction="あなたはあいさつエージェントです。あなたの唯一のタスクは'say_hello'ツールを使用して友好的な挨拶を提供することです。それ以外は何もしないでください。",
        description="'say_hello'ツールを使用して簡単な挨拶を処理します。",
        tools=[say_hello],
    )
    print(f"✅ エージェント '{greeting_agent.name}' が再定義されました。")
except Exception as e:
    print(f"❌ あいさつエージェントを再定義できませんでした。エラー: {e}")

# --- 別れのエージェントの再定義（ステップ3から） ---
farewell_agent = None
try:
    farewell_agent = Agent(
        model=LiteLlm(model=MODEL_GPT_o3_MINI),
        name="farewell_agent",
        instruction="あなたは別れのエージェントです。あなたの唯一のタスクは'say_goodbye'ツールを使用して丁寧な別れのメッセージを提供することです。他のアクションは実行しないでください。",
        description="'say_goodbye'ツールを使用して簡単な別れの挨拶を処理します。",
        tools=[say_goodbye],
    )
    print(f"✅ エージェント '{farewell_agent.name}' が再定義されました。")
except Exception as e:
    print(f"❌ 別れのエージェントを再定義できませんでした。エラー: {e}")

# --- 更新されたルートエージェントの定義 ---
root_agent_stateful = None
runner_root_stateful = None # ランナーの初期化

# ルートエージェントを作成する前に前提条件を確認
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals():

    root_agent_model = LiteLlm(model=MODEL_GPT_o3_MINI) # オーケストレーションモデルの選択

    root_agent_stateful = Agent(
        name="weather_agent_v4_stateful", # 新しいバージョン名
        model=root_agent_model,
        description="メインエージェント: 天気情報を提供（状態認識単位）、挨拶/別れを委任、レポートを状態に保存します。",
        instruction="あなたは主要な天気エージェントです。あなたの仕事は'get_weather_stateful'を使用して天気情報を提供することです。"
                    "このツールは状態に保存されているユーザー設定に基づいて温度をフォーマットします。"
                    "簡単な挨拶は'greeting_agent'に、別れは'farewell_agent'に委任してください。"
                    "天気のリクエスト、挨拶、別れのみを処理してください。",
        tools=[get_weather_stateful], # 状態認識ツールを使用
        sub_agents=[greeting_agent, farewell_agent], # サブエージェントを含める
        output_key="last_weather_report" # <<< エージェントの最終的な天気レスポンスを自動保存
    )
    print(f"✅ ルートエージェント '{root_agent_stateful.name}' が状態認識ツールとoutput_keyを使用して作成されました。")

    # --- このルートエージェントと新しいセッションサービス用のランナーを作成 ---
    runner_root_stateful = Runner(
        agent=root_agent_stateful,
        app_name=APP_NAME,
        session_service=session_service_stateful # 新しい状態認識セッションサービスを使用
    )
    print(f"✅ 状態認識ルートエージェント '{runner_root_stateful.agent.name}' 用のランナーが状態認識セッションサービスを使用して作成されました。")

else:
    print("❌ 状態認識ルートエージェントを作成できません。前提条件が不足しています。")
    if not greeting_agent: print(" - greeting_agent の定義が不足しています。")
    if not farewell_agent: print(" - farewell_agent の定義が不足しています。")
    if 'get_weather_stateful' not in globals(): print(" - get_weather_stateful ツールが不足しています。")


✅ エージェント 'greeting_agent' が再定義されました。
✅ エージェント 'farewell_agent' が再定義されました。
✅ ルートエージェント 'weather_agent_v4_stateful' が状態認識ツールとoutput_keyを使用して作成されました。
✅ 状態認識ルートエージェント 'weather_agent_v4_stateful' 用のランナーが状態認識セッションサービスを使用して作成されました。


 ---
 
 **4\. 状態フローの対話とテスト**
 
 ここでは、`runner_root_stateful`（状態を持つエージェントと`session_service_stateful`に関連付けられている）を使用して、状態の相互作用をテストするための会話を実行します。先ほど定義した`call_agent_async`関数を使用し、正しいランナー、ユーザーID（`USER_ID_STATEFUL`）、セッションID（`SESSION_ID_STATEFUL`）を渡すようにします。
 
 会話の流れは次のとおりです：
 
 1.  **天気の確認（ロンドン）：** `get_weather_stateful`ツールは、セクション1で初期化されたセッション状態から初期の「摂氏」設定を読み取ります。ルートエージェントの最終応答（摂氏での天気レポート）は、`output_key`設定を通じて`state['last_weather_report']`に保存されます。
 2.  **手動での状態更新：** `InMemorySessionService`インスタンス（`session_service_stateful`）内に保存されている状態を*直接変更*します。
     *   **なぜ直接変更するのか？** `session_service.get_session()`メソッドはセッションの*コピー*を返します。そのコピーを変更しても、後続のエージェント実行で使用される状態には影響しません。この`InMemorySessionService`を使用したテストシナリオでは、内部の`sessions`ディクショナリにアクセスして、`user_preference_temperature_unit`の*実際の*保存値を「華氏」に変更します。*注意：実際のアプリケーションでは、状態の変更は通常、直接手動更新ではなく、ツールやエージェントのロジックが`EventActions(state_delta=...)`を返すことによってトリガーされます。*
 3.  **再度天気を確認（ニューヨーク）：** `get_weather_stateful`ツールは、状態から更新された「華氏」設定を読み取り、それに応じて温度を変換します。ルートエージェントの*新しい*応答（華氏での天気）は、`output_key`により`state['last_weather_report']`の以前の値を上書きします。
 4.  **エージェントに挨拶：** 状態を持つ操作と並行して、`greeting_agent`への委任が正しく機能することを確認します。この対話は、この特定のシーケンスで`output_key`によって保存される*最後の*応答になります。
 5.  **最終状態の検査：** 会話の後、セッションを最後にもう一度取得し（コピーを取得）、その状態を出力して`user_preference_temperature_unit`が確かに「華氏」になっていることを確認し、`output_key`によって保存された最終値（この実行では挨拶）を観察し、ツールによって書き込まれた`last_city_checked_stateful`の値を確認します。


In [28]:
# @title 4. 状態フローとoutput_keyをテストするための対話

# 前のセルから状態を持つランナー（runner_root_stateful）が利用可能であることを確認
# call_agent_async、USER_ID_STATEFUL、SESSION_ID_STATEFUL、APP_NAMEが定義されていることを確認

if 'runner_root_stateful' in globals() and runner_root_stateful:
    async def run_stateful_conversation():
        print("\n--- 状態テスト: 温度単位変換とoutput_key ---")

        # 1. 天気を確認（初期状態: 摂氏）
        print("--- ターン1: ロンドンの天気をリクエスト（摂氏を期待）---")
        await call_agent_async(query= "ロンドンの天気はどうですか？",
                                runner=runner_root_stateful,
                                user_id=USER_ID_STATEFUL,
                                session_id=SESSION_ID_STATEFUL
                            )

        # 2. 手動で状態の設定を華氏に更新 - ストレージを直接変更
        print("\n--- 状態を手動で更新: 単位を華氏に設定 ---")
        try:
            # 内部ストレージに直接アクセス - これはテスト用のInMemorySessionServiceに特有のもの
            stored_session = session_service_stateful.sessions[APP_NAME][USER_ID_STATEFUL][SESSION_ID_STATEFUL]
            stored_session.state["user_preference_temperature_unit"] = "Fahrenheit"
            # オプション: ロジックがタイムスタンプに依存する場合は、タイムスタンプも更新するとよいでしょう
            # import time
            # stored_session.last_update_time = time.time()
            print(f"--- 保存されたセッション状態が更新されました。現在の 'user_preference_temperature_unit': {stored_session.state['user_preference_temperature_unit']} ---")
        except KeyError:
            print(f"--- エラー: 状態を更新するために、アプリ '{APP_NAME}' のユーザー '{USER_ID_STATEFUL}' のセッション '{SESSION_ID_STATEFUL}' を内部ストレージから取得できませんでした。IDを確認し、セッションが作成されているか確認してください。 ---")
        except Exception as e:
            print(f"--- 内部セッション状態の更新エラー: {e} ---")

        # 3. 再度天気を確認（ツールは華氏を使用するはず）
        # これはoutput_keyを通じて'last_weather_report'も更新します
        print("\n--- ターン2: ニューヨークの天気をリクエスト（華氏を期待）---")
        await call_agent_async(query= "ニューヨークの天気を教えてください。",
                                runner=runner_root_stateful,
                                user_id=USER_ID_STATEFUL,
                                session_id=SESSION_ID_STATEFUL
                            )

        # 4. 基本的な委任をテスト（まだ機能するはず）
        # これは'last_weather_report'を再度更新し、NYの天気レポートを上書きします
        print("\n--- ターン3: 挨拶を送信 ---")
        await call_agent_async(query= "こんにちは！",
                                runner=runner_root_stateful,
                                user_id=USER_ID_STATEFUL,
                                session_id=SESSION_ID_STATEFUL
                                )

    # 会話を実行
    await run_stateful_conversation()

    # 会話後の最終セッション状態を検査
    print("\n--- 最終セッション状態の検査 ---")
    final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                        user_id= USER_ID_STATEFUL,
                                                        session_id=SESSION_ID_STATEFUL)
    if final_session: 
        print(f"最終設定: {final_session.state.get('user_preference_temperature_unit')}")
        print(f"最終天気レポート（output_keyから）: {final_session.state.get('last_weather_report')}")
        print(f"最終確認都市（ツールによる）: {final_session.state.get('last_city_checked_stateful')}")
        # 詳細表示のために完全な状態を表示
        # print(f"完全な状態: {final_session.state}")
    else:
        print("\n❌ エラー: 最終セッション状態を取得できませんでした。")

else:
    print("\n⚠️ 状態テスト会話をスキップします。状態を持つルートエージェントランナー（'runner_root_stateful'）が利用できません。")


--- 状態テスト: 温度単位変換とoutput_key ---
--- ターン1: ロンドンの天気をリクエスト（摂氏を期待）---

>>> ユーザークエリ: ロンドンの天気はどうですか？
--- ツール: get_weather_stateful が London のために呼び出されました ---
--- ツール: 状態 'user_preference_temperature_unit' を読み取り中: Celsius ---
--- ツール: Celsiusで天気レポートを生成しました。結果: {'status': 'success', 'report': 'Londonの天気はcloudyで、気温は15°Cです。'} ---
--- ツール: 状態 'last_city_checked_stateful' を更新: London ---
<<< エージェント応答: Londonの天気はcloudyで、気温は15°Cです。

--- 状態を手動で更新: 単位を華氏に設定 ---
--- 保存されたセッション状態が更新されました。現在の 'user_preference_temperature_unit': Fahrenheit ---

--- ターン2: ニューヨークの天気をリクエスト（華氏を期待）---

>>> ユーザークエリ: ニューヨークの天気を教えてください。
--- ツール: get_weather_stateful が New York のために呼び出されました ---
--- ツール: 状態 'user_preference_temperature_unit' を読み取り中: Fahrenheit ---
--- ツール: Fahrenheitで天気レポートを生成しました。結果: {'status': 'success', 'report': 'New yorkの天気はsunnyで、気温は77°Fです。'} ---
--- ツール: 状態 'last_city_checked_stateful' を更新: New York ---
<<< エージェント応答: New Yorkの天気はsunnyで、気温は77°Fです。

--- ターン3: 挨拶を送信 ---

>>> ユーザークエリ: こんにちは！
--- ツール: say_hello 

 ---
 
 会話の流れと最終セッション状態の出力を確認することで、以下のことが確認できます：
 
 *   **状態の読み取り：** 天気ツール（`get_weather_stateful`）は状態から`user_preference_temperature_unit`を正しく読み取り、最初はロンドンの天気を「摂氏」で表示しました。
 *   **状態の更新：** 直接的な変更により、保存された設定が「華氏」に正常に変更されました。
 *   **状態の読み取り（更新後）：** ツールはニューヨークの天気を尋ねられたとき、「華氏」を読み取り、変換を実行しました。
 *   **ツールによる状態の書き込み：** ツールは`tool_context.state`を通じて`last_city_checked_stateful`（2回目の天気確認後の「ニューヨーク」）を状態に正常に書き込みました。
 *   **委任：** 状態が変更された後でも、「こんにちは！」に対する`greeting_agent`への委任は正常に機能しました。
 *   **`output_key`：** `output_key="last_weather_report"`は、ルートエージェントが最終的に応答した*各ターン*の*最終*応答を正常に保存しました。このシーケンスでは、最後の応答は挨拶（「こんにちは！」）だったため、状態キー内の天気レポートが上書きされました。
 *   **最終状態：** 最終確認により、設定が「華氏」として維持されていることが確認されました。
 
 これで、`ToolContext`を使用してエージェントの動作をパーソナライズするためのセッション状態の統合、`InMemorySessionService`をテストするための状態の手動操作、そして`output_key`がエージェントの最後の応答を状態に保存するためのシンプルなメカニズムを提供する方法を確認できました。この状態管理の基本的な理解は、次のステップでコールバックを使用して安全性ガードレールを実装する際に重要となります。
 
 ---

# ## ステップ5: 安全性の追加 - `before_model_callback`を使用した入力ガードレール

私たちのエージェントチームは、設定を記憶しツールを効果的に使用するなど、より高機能になってきています。しかし、実際のシナリオでは、潜在的に問題のあるリクエストがコアの大規模言語モデル（LLM）に到達する*前に*、エージェントの動作を制御するための安全メカニズムが必要になることがよくあります。

ADKは**コールバック**を提供しています - エージェントの実行ライフサイクルの特定のポイントにフックできる関数です。特に`before_model_callback`は入力の安全性に非常に役立ちます。

**`before_model_callback`とは？**

* エージェントがコンパイルされたリクエスト（会話履歴、指示、最新のユーザーメッセージを含む）を基盤となるLLMに送信する*直前に*、ADKが実行するあなたが定義するPython関数です。
* **目的:** リクエストを検査し、必要に応じて変更したり、事前定義されたルールに基づいて完全にブロックしたりします。

**一般的なユースケース:**

* **入力の検証/フィルタリング:** ユーザー入力が基準を満たしているか、または許可されていないコンテンツ（PIIやキーワードなど）を含んでいないかを確認します。
* **ガードレール:** 有害、トピック外、またはポリシー違反のリクエストがLLMによって処理されるのを防ぎます。
* **動的プロンプト修正:** 送信直前にLLMリクエストコンテキストにタイムリーな情報（セッション状態からなど）を追加します。

**仕組み:**

1. `callback_context: CallbackContext`と`llm_request: LlmRequest`を受け入れる関数を定義します。
   * `callback_context`: エージェント情報、セッション状態（`callback_context.state`）などへのアクセスを提供します。
   * `llm_request`: LLM向けの完全なペイロード（`contents`、`config`）を含みます。
2. 関数内で:
   * **検査:** `llm_request.contents`（特に最後のユーザーメッセージ）を調べます。
   * **修正（注意して使用）:** `llm_request`の一部を変更*できます*。
   * **ブロック（ガードレール）:** `LlmResponse`オブジェクトを返します。ADKはこの応答をすぐに送り返し、そのターンのLLM呼び出しを*スキップ*します。
   * **許可:** `None`を返します。ADKは（潜在的に修正された）リクエストでLLMを呼び出します。

**このステップでは:**

1. ユーザーの入力に特定のキーワード（「BLOCK」）がないかチェックする`before_model_callback`関数（`block_keyword_guardrail`）を定義します。
2. ステップ4の状態を持つルートエージェント（`weather_agent_v4_stateful`）を更新して、このコールバックを使用します。
3. この更新されたエージェントに関連付けられた新しいランナーを作成しますが、状態の継続性を維持するために*同じ状態を持つセッションサービス*を使用します。
4. 通常のリクエストとキーワードを含むリクエストの両方を送信して、ガードレールをテストします。

 ---
 
 **1\. ガードレールコールバック関数の定義**
 
 この関数は`llm_request`コンテンツ内の最後のユーザーメッセージを検査します。「BLOCK」（大文字小文字を区別しない）が見つかった場合、フローをブロックするために`LlmResponse`を構築して返します。それ以外の場合は`None`を返します。

In [ ]:
# @title 1. before_model_callback ガードレールの定義

# 必要なインポートを確保
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.genai import types # レスポンスコンテンツの作成用
from typing import Optional

def block_keyword_guardrail(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """
    最新のユーザーメッセージに「BLOCK」が含まれているか検査します。見つかった場合、
    LLM呼び出しをブロックし、事前定義されたLlmResponseを返します。それ以外の場合はNoneを返して処理を続行します。
    """
    agent_name = callback_context.agent_name # インターセプトされているエージェントの名前を取得
    print(f"--- コールバック: block_keyword_guardrail が実行中 エージェント: {agent_name} ---")

    # リクエスト履歴から最新のユーザーメッセージのテキストを抽出
    last_user_message_text = ""
    if llm_request.contents:
        # ロールが「user」の最新のメッセージを検索
        for content in reversed(llm_request.contents):
            if content.role == 'user' and content.parts:
                # 簡略化のため、テキストは最初のパートにあると仮定
                if content.parts[0].text:
                    last_user_message_text = content.parts[0].text
                    break # 最後のユーザーメッセージテキストを見つけた

    print(f"--- コールバック: 最後のユーザーメッセージを検査中: '{last_user_message_text[:100]}...' ---") # 最初の100文字をログに記録

    # --- ガードレールロジック ---
    keyword_to_block = "BLOCK"
    if keyword_to_block in last_user_message_text.upper(): # 大文字小文字を区別しないチェック
        print(f"--- コールバック: '{keyword_to_block}'が見つかりました。LLM呼び出しをブロックします！ ---")
        # オプションで、ブロックイベントを記録するためにステートにフラグを設定
        callback_context.state["guardrail_block_keyword_triggered"] = True
        print(f"--- コールバック: ステート 'guardrail_block_keyword_triggered': True を設定しました ---")

        # フローを停止し、代わりにこれを送信するためのLlmResponseを構築して返す
        return LlmResponse(
            content=types.Content(
                role="model", # エージェントの視点からの応答を模倣
                parts=[types.Part(text=f"このリクエストには禁止キーワード '{keyword_to_block}' が含まれているため、処理できません。")],
            )
            # 注：必要に応じてここにerror_messageフィールドを設定することもできます
        )
    else:
        # キーワードが見つからなかった場合、リクエストをLLMに進めることを許可
        print(f"--- コールバック: キーワードが見つかりませんでした。{agent_name}のLLM呼び出しを許可します。 ---")
        return None # Noneを返すことで、ADKに通常通り続行するよう指示

print("✅ block_keyword_guardrail関数が定義されました。")


✅ block_keyword_guardrail関数が定義されました。


 ---
 
 **2\. コールバックを使用するためのルートエージェントの更新**
 
 ルートエージェントを再定義し、`before_model_callback`パラメータを追加して、新しいガードレール関数を指定します。わかりやすくするために、新しいバージョン名を付けます。
 

 *重要:* サブエージェント（`greeting_agent`、`farewell_agent`）とステートフルツール（`get_weather_stateful`）が前のステップからすでに利用可能でない場合は、このコンテキスト内で再定義する必要があります。これにより、ルートエージェントの定義がすべてのコンポーネントにアクセスできるようになります。

In [25]:
# LiteLlmを使ったシンプルな対話
# LiteLlmモデルを初期化
# LiteLlmはcompletion()メソッドを持っていないため、
# 正しいメソッドを使用する必要があります
llm = LiteLlm(model=MODEL_GPT_o3_MINI)

# APIリクエストを送信してレスポンスを取得
# LiteLlmではcomplete()メソッドを使用
response = llm.complete(
    prompt="こんにちは、今日の天気を教えてください。",
    temperature=0.7,
    max_tokens=100
)
print("APIレスポンス:", response.choices[0].message.content)

AttributeError: 'LiteLlm' object has no attribute 'complete'

In [20]:
# @title 2. コールバック付きルートエージェントの更新


# --- サブエージェントの再定義（このコンテキストで存在することを確認） ---
greeting_agent = None
try:
    # 定義済みのモデル定数を使用
    greeting_agent = Agent(
        model=LiteLlm(model=MODEL_GPT_o3_MINI),
        name="greeting_agent", # 一貫性のために元の名前を維持
        instruction="あなたは挨拶エージェントです。あなたの唯一のタスクは「say_hello」ツールを使用して友好的な挨拶を提供することです。それ以外は何もしないでください。",
        description="「say_hello」ツールを使用して簡単な挨拶を処理します。",
        tools=[say_hello],
    )
    print(f"✅ サブエージェント '{greeting_agent.name}' が再定義されました。")
except Exception as e:
    print(f"❌ 挨拶エージェントを再定義できませんでした。モデル/APIキー({MODEL_GPT_o3_MINI})を確認してください。エラー: {e}")

farewell_agent = None
try:
    # 定義済みのモデル定数を使用
    farewell_agent = Agent(
        model=LiteLlm(model=MODEL_GPT_o3_MINI),
        name="farewell_agent", # 元の名前を維持
        instruction="あなたは別れのエージェントです。あなたの唯一のタスクは「say_goodbye」ツールを使用して丁寧な別れのメッセージを提供することです。他のアクションは実行しないでください。",
        description="「say_goodbye」ツールを使用して簡単な別れの挨拶を処理します。",
        tools=[say_goodbye],
    )
    print(f"✅ サブエージェント '{farewell_agent.name}' が再定義されました。")
except Exception as e:
    print(f"❌ 別れのエージェントを再定義できませんでした。モデル/APIキー({MODEL_GPT_o3_MINI})を確認してください。エラー: {e}")


# --- コールバック付きのルートエージェントを定義 ---
root_agent_model_guardrail = None
runner_root_model_guardrail = None

# 進める前にすべてのコンポーネントを確認
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals() and 'block_keyword_guardrail' in globals():

    # MODEL_GEMINI_2_5_PROのような定義済みモデル定数を使用
    root_agent_model = LiteLlm(model=MODEL_GPT_o3_MINI)

    root_agent_model_guardrail = Agent(
        name="weather_agent_v5_model_guardrail", # 明確にするための新バージョン名
        model=root_agent_model,
        description="メインエージェント：天気を処理し、挨拶/別れを委任し、入力キーワードガードレールを含みます。",
        instruction="あなたは主要な天気エージェントです。「get_weather_stateful」を使用して天気情報を提供します。"
                    "簡単な挨拶は「greeting_agent」に、別れは「farewell_agent」に委任します。"
                    "天気のリクエスト、挨拶、別れのみを処理してください。",
        tools=[get_weather],
        sub_agents=[greeting_agent, farewell_agent], # 再定義されたサブエージェントを参照
        output_key="last_weather_report", # ステップ4からoutput_keyを維持
        before_model_callback=block_keyword_guardrail # <<< ガードレールコールバックを割り当て
    )
    print(f"✅ ルートエージェント '{root_agent_model_guardrail.name}' がbefore_model_callbackで作成されました。")

    # --- このエージェント用のランナーを作成、同じステートフルセッションサービスを使用 ---
    # ステップ4からsession_service_statefulが存在することを確認
    if 'session_service_stateful' in globals():
        runner_root_model_guardrail = Runner(
            agent=root_agent_model_guardrail,
            app_name=APP_NAME, # 一貫したAPP_NAMEを使用
            session_service=session_service_stateful # <<< ステップ4からのサービスを使用
        )
        print(f"✅ ガードレールエージェント '{runner_root_model_guardrail.agent.name}' 用のランナーが作成され、ステートフルセッションサービスを使用しています。")
    else:
        print("❌ ランナーを作成できません。ステップ4からの 'session_service_stateful' がありません。")

else:
    print("❌ モデルガードレール付きのルートエージェントを作成できません。前提条件の1つ以上が欠落しているか、初期化に失敗しました：")
    if not greeting_agent: print("   - 挨拶エージェント")
    if not farewell_agent: print("   - 別れのエージェント")
    if 'get_weather_stateful' not in globals(): print("   - 'get_weather_stateful' ツール")
    if 'block_keyword_guardrail' not in globals(): print("   - 'block_keyword_guardrail' コールバック")

❌ 挨拶エージェントを再定義できませんでした。モデル/APIキー(openai/o3-mini-2025-01-31)を確認してください。エラー: name 'say_hello' is not defined
❌ 別れのエージェントを再定義できませんでした。モデル/APIキー(openai/o3-mini-2025-01-31)を確認してください。エラー: name 'say_goodbye' is not defined
❌ モデルガードレール付きのルートエージェントを作成できません。前提条件の1つ以上が欠落しているか、初期化に失敗しました：
   - 挨拶エージェント
   - 別れのエージェント
   - 'get_weather_stateful' ツール


---

**3\. Interact to Test the Guardrail**

Let's test the guardrail's behavior. We'll use the *same session* (`SESSION_ID_STATEFUL`) as in Step 4 to show that state persists across these changes.

1. Send a normal weather request (should pass the guardrail and execute).  
2. Send a request containing "BLOCK" (should be intercepted by the callback).  
3. Send a greeting (should pass the root agent's guardrail, be delegated, and execute normally).

In [ ]:
# @title 3. Interact to Test the Model Input Guardrail

# Ensure the runner for the guardrail agent is available
if runner_root_model_guardrail:
  async def run_guardrail_test_conversation():
      print("\n--- Testing Model Input Guardrail ---")

      # Use the runner for the agent with the callback and the existing stateful session ID
      interaction_func = lambda query: call_agent_async(query,
      runner_root_model_guardrail, USER_ID_STATEFUL, SESSION_ID_STATEFUL # <-- Pass correct IDs
  )
      # 1. Normal request (Callback allows, should use Fahrenheit from Step 4 state change)
      await interaction_func("What is the weather in London?")

      # 2. Request containing the blocked keyword
      await interaction_func("BLOCK the request for weather in Tokyo")

      # 3. Normal greeting (Callback allows root agent, delegation happens)
      await interaction_func("Hello again")


  # Execute the conversation
  await run_guardrail_test_conversation()

  # Optional: Check state for the trigger flag set by the callback
  final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                       user_id=USER_ID_STATEFUL,
                                                       session_id=SESSION_ID_STATEFUL)
  if final_session:
      print("\n--- Final Session State (After Guardrail Test) ---")
      print(f"Guardrail Triggered Flag: {final_session.state.get('guardrail_block_keyword_triggered')}")
      print(f"Last Weather Report: {final_session.state.get('last_weather_report')}") # Should be London weather
      print(f"Temperature Unit: {final_session.state.get('user_preference_temperature_unit')}") # Should be Fahrenheit
  else:
      print("\n❌ Error: Could not retrieve final session state.")

else:
  print("\n⚠️ Skipping model guardrail test. Runner ('runner_root_model_guardrail') is not available.")



---

Observe the execution flow:

1. **London Weather:** The callback runs for `weather_agent_v5_model_guardrail`, inspects the message, prints "Keyword not found. Allowing LLM call.", and returns `None`. The agent proceeds, calls the `get_weather_stateful` tool (which uses the "Fahrenheit" preference from Step 4's state change), and returns the weather. This response updates `last_weather_report` via `output_key`.  
2. **BLOCK Request:** The callback runs again for `weather_agent_v5_model_guardrail`, inspects the message, finds "BLOCK", prints "Blocking LLM call\!", sets the state flag, and returns the predefined `LlmResponse`. The agent's underlying LLM is *never called* for this turn. The user sees the callback's blocking message.  
3. **Hello Again:** The callback runs for `weather_agent_v5_model_guardrail`, allows the request. The root agent then delegates to `greeting_agent`. *Note: The `before_model_callback` defined on the root agent does NOT automatically apply to sub-agents.* The `greeting_agent` proceeds normally, calls its `say_hello` tool, and returns the greeting.

You have successfully implemented an input safety layer\! The `before_model_callback` provides a powerful mechanism to enforce rules and control agent behavior *before* expensive or potentially risky LLM calls are made. Next, we'll apply a similar concept to add guardrails around tool usage itself.

## Step 6: Adding Safety \- Tool Argument Guardrail (`before_tool_callback`)

In Step 5, we added a guardrail to inspect and potentially block user input *before* it reached the LLM. Now, we'll add another layer of control *after* the LLM has decided to use a tool but *before* that tool actually executes. This is useful for validating the *arguments* the LLM wants to pass to the tool.

ADK provides the `before_tool_callback` for this precise purpose.

**What is `before_tool_callback`?**

* It's a Python function executed just *before* a specific tool function runs, after the LLM has requested its use and decided on the arguments.  
* **Purpose:** Validate tool arguments, prevent tool execution based on specific inputs, modify arguments dynamically, or enforce resource usage policies.

**Common Use Cases:**

* **Argument Validation:** Check if arguments provided by the LLM are valid, within allowed ranges, or conform to expected formats.  
* **Resource Protection:** Prevent tools from being called with inputs that might be costly, access restricted data, or cause unwanted side effects (e.g., blocking API calls for certain parameters).  
* **Dynamic Argument Modification:** Adjust arguments based on session state or other contextual information before the tool runs.

**How it Works:**

1. Define a function accepting `tool: BaseTool`, `args: Dict[str, Any]`, and `tool_context: ToolContext`.  
   * `tool`: The tool object about to be called (inspect `tool.name`).  
   * `args`: The dictionary of arguments the LLM generated for the tool.  
   * `tool_context`: Provides access to session state (`tool_context.state`), agent info, etc.  
2. Inside the function:  
   * **Inspect:** Examine the `tool.name` and the `args` dictionary.  
   * **Modify:** Change values within the `args` dictionary *directly*. If you return `None`, the tool runs with these modified args.  
   * **Block/Override (Guardrail):** Return a **dictionary**. ADK treats this dictionary as the *result* of the tool call, completely *skipping* the execution of the original tool function. The dictionary should ideally match the expected return format of the tool it's blocking.  
   * **Allow:** Return `None`. ADK proceeds to execute the actual tool function with the (potentially modified) arguments.

**In this step, we will:**

1. Define a `before_tool_callback` function (`block_paris_tool_guardrail`) that specifically checks if the `get_weather_stateful` tool is called with the city "Paris".  
2. If "Paris" is detected, the callback will block the tool and return a custom error dictionary.  
3. Update our root agent (`weather_agent_v6_tool_guardrail`) to include *both* the `before_model_callback` and this new `before_tool_callback`.  
4. Create a new runner for this agent, using the same stateful session service.  
5. Test the flow by requesting weather for allowed cities and the blocked city ("Paris").

---

**1\. Define the Tool Guardrail Callback Function**

This function targets the `get_weather_stateful` tool. It checks the `city` argument. If it's "Paris", it returns an error dictionary that looks like the tool's own error response. Otherwise, it allows the tool to run by returning `None`.

In [ ]:
# @title 1. Define the before_tool_callback Guardrail

# Ensure necessary imports are available
from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
from typing import Optional, Dict, Any # For type hints

def block_paris_tool_guardrail(
    tool: BaseTool, args: Dict[str, Any], tool_context: ToolContext
) -> Optional[Dict]:
    """
    Checks if 'get_weather_stateful' is called for 'Paris'.
    If so, blocks the tool execution and returns a specific error dictionary.
    Otherwise, allows the tool call to proceed by returning None.
    """
    tool_name = tool.name
    agent_name = tool_context.agent_name # Agent attempting the tool call
    print(f"--- Callback: block_paris_tool_guardrail running for tool '{tool_name}' in agent '{agent_name}' ---")
    print(f"--- Callback: Inspecting args: {args} ---")

    # --- Guardrail Logic ---
    target_tool_name = "get_weather_stateful" # Match the function name used by FunctionTool
    blocked_city = "paris"

    # Check if it's the correct tool and the city argument matches the blocked city
    if tool_name == target_tool_name:
        city_argument = args.get("city", "") # Safely get the 'city' argument
        if city_argument and city_argument.lower() == blocked_city:
            print(f"--- Callback: Detected blocked city '{city_argument}'. Blocking tool execution! ---")
            # Optionally update state
            tool_context.state["guardrail_tool_block_triggered"] = True
            print(f"--- Callback: Set state 'guardrail_tool_block_triggered': True ---")

            # Return a dictionary matching the tool's expected output format for errors
            # This dictionary becomes the tool's result, skipping the actual tool run.
            return {
                "status": "error",
                "error_message": f"Policy restriction: Weather checks for '{city_argument.capitalize()}' are currently disabled by a tool guardrail."
            }
        else:
             print(f"--- Callback: City '{city_argument}' is allowed for tool '{tool_name}'. ---")
    else:
        print(f"--- Callback: Tool '{tool_name}' is not the target tool. Allowing. ---")


    # If the checks above didn't return a dictionary, allow the tool to execute
    print(f"--- Callback: Allowing tool '{tool_name}' to proceed. ---")
    return None # Returning None allows the actual tool function to run

print("✅ block_paris_tool_guardrail function defined.")



---

**2\. Update Root Agent to Use Both Callbacks**

We redefine the root agent again (`weather_agent_v6_tool_guardrail`), this time adding the `before_tool_callback` parameter alongside the `before_model_callback` from Step 5\.

*Self-Contained Execution Note:* Similar to Step 5, ensure all prerequisites (sub-agents, tools, `before_model_callback`) are defined or available in the execution context before defining this agent.

In [ ]:
# @title 2. Update Root Agent with BOTH Callbacks (Self-Contained)

# --- Ensure Prerequisites are Defined ---
# (Include or ensure execution of definitions for: Agent, LiteLlm, Runner, ToolContext,
#  MODEL constants, say_hello, say_goodbye, greeting_agent, farewell_agent,
#  get_weather_stateful, block_keyword_guardrail, block_paris_tool_guardrail)

# --- Redefine Sub-Agents (Ensures they exist in this context) ---
greeting_agent = None
try:
    # Use a defined model constant like MODEL_GPT_o3_MINI
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent", # Keep original name for consistency
        instruction="You are the Greeting Agent. Your ONLY task is to provide a friendly greeting using the 'say_hello' tool. Do nothing else.",
        description="Handles simple greetings and hellos using the 'say_hello' tool.",
        tools=[say_hello],
    )
    print(f"✅ Sub-Agent '{greeting_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Greeting agent. Check Model/API Key ({MODEL_GPT_o3_MINI}). Error: {e}")

farewell_agent = None
try:
    # Use a defined model constant like MODEL_GPT_o3_MINI
    farewell_agent = Agent(
        model=LiteLlm(model=MODEL_GPT_o3_MINI),
        name="farewell_agent", # Keep original name
        instruction="You are the Farewell Agent. Your ONLY task is to provide a polite goodbye message using the 'say_goodbye' tool. Do not perform any other actions.",
        description="Handles simple farewells and goodbyes using the 'say_goodbye' tool.",
        tools=[say_goodbye],
    )
    print(f"✅ Sub-Agent '{farewell_agent.name}' redefined.")
except Exception as e:
    print(f"❌ Could not redefine Farewell agent. Check Model/API Key ({MODEL_GPT_o3_MINI}). Error: {e}")

# --- Define the Root Agent with Both Callbacks ---
root_agent_tool_guardrail = None
runner_root_tool_guardrail = None

if ('greeting_agent' in globals() and greeting_agent and
    'farewell_agent' in globals() and farewell_agent and
    'get_weather_stateful' in globals() and
    'block_keyword_guardrail' in globals() and
    'block_paris_tool_guardrail' in globals()):

    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_tool_guardrail = Agent(
        name="weather_agent_v6_tool_guardrail", # New version name
        model=root_agent_model,
        description="Main agent: Handles weather, delegates, includes input AND tool guardrails.",
        instruction="You are the main Weather Agent. Provide weather using 'get_weather_stateful'. "
                    "Delegate greetings to 'greeting_agent' and farewells to 'farewell_agent'. "
                    "Handle only weather, greetings, and farewells.",
        tools=[get_weather_stateful],
        sub_agents=[greeting_agent, farewell_agent],
        output_key="last_weather_report",
        before_model_callback=block_keyword_guardrail, # Keep model guardrail
        before_tool_callback=block_paris_tool_guardrail # <<< Add tool guardrail
    )
    print(f"✅ Root Agent '{root_agent_tool_guardrail.name}' created with BOTH callbacks.")

    # --- Create Runner, Using SAME Stateful Session Service ---
    if 'session_service_stateful' in globals():
        runner_root_tool_guardrail = Runner(
            agent=root_agent_tool_guardrail,
            app_name=APP_NAME,
            session_service=session_service_stateful # <<< Use the service from Step 4/5
        )
        print(f"✅ Runner created for tool guardrail agent '{runner_root_tool_guardrail.agent.name}', using stateful session service.")
    else:
        print("❌ Cannot create runner. 'session_service_stateful' from Step 4/5 is missing.")

else:
    print("❌ Cannot create root agent with tool guardrail. Prerequisites missing.")



---

**3\. Interact to Test the Tool Guardrail**

Let's test the interaction flow, again using the same stateful session (`SESSION_ID_STATEFUL`) from the previous steps.

1. Request weather for "New York": Passes both callbacks, tool executes (using Fahrenheit preference from state).  
2. Request weather for "Paris": Passes `before_model_callback`. LLM decides to call `get_weather_stateful(city='Paris')`. `before_tool_callback` intercepts, blocks the tool, and returns the error dictionary. Agent relays this error.  
3. Request weather for "London": Passes both callbacks, tool executes normally.

In [ ]:
# @title 3. Interact to Test the Tool Argument Guardrail

# Ensure the runner for the tool guardrail agent is available
if runner_root_tool_guardrail:
  async def run_tool_guardrail_test():
      print("\n--- Testing Tool Argument Guardrail ('Paris' blocked) ---")

        # Use the runner for the agent with both callbacks and the existing stateful session
      interaction_func = lambda query: call_agent_async(query,
      runner_root_tool_guardrail, USER_ID_STATEFUL, SESSION_ID_STATEFUL
  )
      # 1. Allowed city (Should pass both callbacks, use Fahrenheit state)
      await interaction_func("What's the weather in New York?")

      # 2. Blocked city (Should pass model callback, but be blocked by tool callback)
      await interaction_func("How about Paris?")

      # 3. Another allowed city (Should work normally again)
      await interaction_func("Tell me the weather in London.")

  # Execute the conversation
  await run_tool_guardrail_test()

  # Optional: Check state for the tool block trigger flag
  final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                       user_id=USER_ID_STATEFUL,
                                                       session_id= SESSION_ID_STATEFUL)
  if final_session:
      print("\n--- Final Session State (After Tool Guardrail Test) ---")
      print(f"Tool Guardrail Triggered Flag: {final_session.state.get('guardrail_tool_block_triggered')}")
      print(f"Last Weather Report: {final_session.state.get('last_weather_report')}") # Should be London weather
      print(f"Temperature Unit: {final_session.state.get('user_preference_temperature_unit')}") # Should be Fahrenheit
  else:
      print("\n❌ Error: Could not retrieve final session state.")

else:
  print("\n⚠️ Skipping tool guardrail test. Runner ('runner_root_tool_guardrail') is not available.")

---

Analyze the output:

1. **New York:** The `before_model_callback` allows the request. The LLM requests `get_weather_stateful`. The `before_tool_callback` runs, inspects the args (`{'city': 'New York'}`), sees it's not "Paris", prints "Allowing tool..." and returns `None`. The actual `get_weather_stateful` function executes, reads "Fahrenheit" from state, and returns the weather report. The agent relays this, and it gets saved via `output_key`.  
2. **Paris:** The `before_model_callback` allows the request. The LLM requests `get_weather_stateful(city='Paris')`. The `before_tool_callback` runs, inspects the args, detects "Paris", prints "Blocking tool execution\!", sets the state flag, and returns the error dictionary `{'status': 'error', 'error_message': 'Policy restriction...'}`. The actual `get_weather_stateful` function is **never executed**. The agent receives the error dictionary *as if it were the tool's output* and formulates a response based on that error message.  
3. **London:** Behaves like New York, passing both callbacks and executing the tool successfully. The new London weather report overwrites the `last_weather_report` in the state.

You've now added a crucial safety layer controlling not just *what* reaches the LLM, but also *how* the agent's tools can be used based on the specific arguments generated by the LLM. Callbacks like `before_model_callback` and `before_tool_callback` are essential for building robust, safe, and policy-compliant agent applications.



---


## Conclusion: Your Agent Team is Ready!

Congratulations! You've successfully journeyed from building a single, basic weather agent to constructing a sophisticated, multi-agent team using the Agent Development Kit (ADK).

**Let's recap what you've accomplished:**

*   You started with a **fundamental agent** equipped with a single tool (`get_weather`).
*   You explored ADK's **multi-model flexibility** using LiteLLM, running the same core logic with different LLMs like Gemini, GPT-4o, and Claude.
*   You embraced **modularity** by creating specialized sub-agents (`greeting_agent`, `farewell_agent`) and enabling **automatic delegation** from a root agent.
*   You gave your agents **memory** using **Session State**, allowing them to remember user preferences (`temperature_unit`) and past interactions (`output_key`).
*   You implemented crucial **safety guardrails** using both `before_model_callback` (blocking specific input keywords) and `before_tool_callback` (blocking tool execution based on arguments like the city "Paris").

Through building this progressive Weather Bot team, you've gained hands-on experience with core ADK concepts essential for developing complex, intelligent applications.

**Key Takeaways:**

*   **Agents & Tools:** The fundamental building blocks for defining capabilities and reasoning. Clear instructions and docstrings are paramount.
*   **Runners & Session Services:** The engine and memory management system that orchestrate agent execution and maintain conversational context.
*   **Delegation:** Designing multi-agent teams allows for specialization, modularity, and better management of complex tasks. Agent `description` is key for auto-flow.
*   **Session State (`ToolContext`, `output_key`):** Essential for creating context-aware, personalized, and multi-turn conversational agents.
*   **Callbacks (`before_model`, `before_tool`):** Powerful hooks for implementing safety, validation, policy enforcement, and dynamic modifications *before* critical operations (LLM calls or tool execution).
*   **Flexibility (`LiteLlm`):** ADK empowers you to choose the best LLM for the job, balancing performance, cost, and features.

**Where to Go Next?**

Your Weather Bot team is a great starting point. Here are some ideas to further explore ADK and enhance your application:

1.  **Real Weather API:** Replace the `mock_weather_db` in your `get_weather` tool with a call to a real weather API (like OpenWeatherMap, WeatherAPI).
2.  **More Complex State:** Store more user preferences (e.g., preferred location, notification settings) or conversation summaries in the session state.
3.  **Refine Delegation:** Experiment with different root agent instructions or sub-agent descriptions to fine-tune the delegation logic. Could you add a "forecast" agent?
4.  **Advanced Callbacks:**
    *   Use `after_model_callback` to potentially reformat or sanitize the LLM's response *after* it's generated.
    *   Use `after_tool_callback` to process or log the results returned by a tool.
    *   Implement `before_agent_callback` or `after_agent_callback` for agent-level entry/exit logic.
5.  **Error Handling:** Improve how the agent handles tool errors or unexpected API responses. Maybe add retry logic within a tool.
6.  **Persistent Session Storage:** Explore alternatives to `InMemorySessionService` for storing session state persistently (e.g., using databases like Firestore or Cloud SQL – requires custom implementation or future ADK integrations).
7.  **Streaming UI:** Integrate your agent team with a web framework (like FastAPI, as shown in the ADK Streaming Quickstart) to create a real-time chat interface.

The Agent Development Kit provides a robust foundation for building sophisticated LLM-powered applications. By mastering the concepts covered in this tutorial – tools, state, delegation, and callbacks – you are well-equipped to tackle increasingly complex agentic systems.

Happy building!